not done.

the model can't predict never seen data.

In [1]:
import numpy as np
from scipy.stats import skew

from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

import pandas as pd

In [2]:
# expected = 0
print(skew(a= [[-1],
               [0],
               [1]],
           axis=0))

[0.]


In [3]:
#              X1    y
X_train = np.array([
              [0], # 0
              [0], # 0
              [0], # 0
              [1], # 1
              [1], # 1
              [1], # 1
              [2], # 0
              [2], # 0
              [2], # 0
              [3], # 1
              [3], # 1
              [3], # 1
              [4], # 0
              [4], # 0
              [4], # 0
              [5], # 1
              [5], # 1
              [5], # 1
              [6], # 0
              [6], # 0
              [6], # 0
              [7], # 1
              [7], # 1
              [7], # 1
             ]
            )

# skew is 0, can use MinMaxScaler
# print(skew(a= X,
#            axis=0))

# change to degree=2 to get lower recall_score
polyer = PolynomialFeatures(degree=3).fit(X= X_train)

X_polyed = polyer.transform(X= X_train)

scaler = MinMaxScaler().fit(X= X_polyed)

X_train_scaled = scaler.transform(X= X_polyed)

#             X1
y_train = [
     0,0,0, # 0
     1,1,1, # 1
     0,0,0, # 2
     1,1,1, # 3
     0,0,0, # 4
     1,1,1, # 5
     0,0,0, # 6
     1,1,1, # 7
    ]

clf = LogisticRegression()
clf.fit(X= X_train_scaled, y= y_train)

# seen data
# X_test = np.array([[4],
#                    [4],
#                    [4],
#                    [5],
#                    [5],
#                    [5]])

# never seen data
X_test = np.array([[8],
                   [8],
                   [8],
                   [9],
                   [9],
                   [9]])

y_test = np.array([0,
                   0,
                   0,
                   1,
                   1,
                   1])

X_test_polyed = polyer.transform(X= X_test)

X_test_scaled = scaler.transform(X= X_test_polyed)

y_test_pred = clf.predict(X= X_test_scaled)

print("y_true", y_test)
print("y_pred", y_test_pred)

# average=None is important.
# otherwise you get average='binary', pos_label=1
print("accuracy score", accuracy_score(y_true=y_test, y_pred=y_test_pred))
print("recall score", recall_score(y_true=y_test, y_pred=y_test_pred))
print("precision score", precision_score(y_true=y_test, y_pred=y_test_pred, average=None, zero_division=0))
print("f1 score", f1_score(y_true=y_test, y_pred=y_test_pred, average=None))


y_true [0 0 0 1 1 1]
y_pred [1 1 1 1 1 1]
accuracy score 0.5
recall score 1.0
precision score [0.  0.5]
f1 score [0.         0.66666667]


In [16]:
def cross_validate_custom(estimator: dict):
    scoring = {'recall': make_scorer(score_func=recall_score, pos_label=1),
               'precision': make_scorer(score_func=precision_score, pos_label=1, zero_division=0),
               'f1': make_scorer(score_func=f1_score, pos_label=1),
               'accuracy_score': make_scorer(score_func=accuracy_score)}
    cv_result = cross_validate(estimator=estimator, X=X_train_scaled, y=y_train,
                               scoring=scoring)

    get_mean_std = lambda array: [array.mean(), array.std()]

    pd.options.display.float_format

    with pd.option_context("display.float_format", "{:.2f}".format):
        display(pd.DataFrame(data={'recall': get_mean_std(cv_result['test_recall']),
                                   'precision': get_mean_std(cv_result['test_precision']),
                                   'f1': get_mean_std(cv_result['test_f1']),
                                   'accuracy_score': get_mean_std(cv_result['test_accuracy_score'])
                                  },
                            index=['mean', 'std']))
    
cross_validate_custom(estimator=clf)

,recall,precision,f1,accuracy_score
mean,0.47,0.45,0.40,0.62
std,0.45,0.40,0.35,0.10
